In [6]:
from pororo import Pororo
import pandas as pd
import torch

In [7]:
Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s

In [8]:
origin_data = pd.read_csv("./data/strongbuy.csv")

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [11]:
mt = Pororo(task="translation", lang="multi")

In [ ]:
def back_trans_pororo(original_text, lang='en'):
    text_to_lang = mt(original_text, src="ko", tgt=lang)
    new_text = mt(text_to_lang, src=lang, tgt="ko")
    return new_text

In [12]:
original_text = "안녕하세요 선생님, 안녕 친구야"
original_text

'안녕하세요 선생님, 안녕 친구야'

In [13]:
mt("안녕하세요 선생님 안녕 친구야", src="ko", tgt="en")

2022-08-10 13:19:16 | WARNING | root | Initializing Pynori...


TypeError: __init__() got an unexpected keyword argument 'persistent_workers'